In [2]:
# from session 1 of advanced predictive modeling

from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import math as math
%matplotlib inline

In [3]:
# https://github.com/magee256/TurbomoleScripts/blob/master/SetUp/add_ligand/add_ligand.py

# add arguments for original coordinates, and both components of principle_comps, 
def rotate_frag(coordinates, rots):
    """
    Returns coordinates rotated counterclockwise about 
    a rotation axis, with both degree rotated and axis direction 
    specified by the Euler-Rodrigues formula vector parameters in rots.
    
    Parameters
    ----------
    coordinates: numpy array of shape (N,3)
    rots: numpy array of shape (_,_) , representing coefficients of axis of rotation * sin(theta/2), see formula
    
    Returns
    -------
    Transformed `coordinates` rotated about the rotation axis `rots`
    
    """
    aa = 1 - np.dot(rots,rots)
    print "aa is %f \n" % aa
    a = math.sqrt(aa)
    print "a is %f \n" % a
    bb, cc, dd = rots[0]*rots[0], rots[1]*rots[1], rots[2]*rots[2]
    bc, ad, ac, ab, bd, cd = rots[0]*rots[1], a*rots[2], a*rots[1], \
                             a*rots[0], rots[0]*rots[2], rots[1]*rots[2]
    r_mat = np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                    [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                    [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

    return np.dot(coordinates,r_mat.T)

In [4]:
# load in coordinates
rough=np.array(
[[-2.00861,   2.12329,   2.71423],
 [ -2.13619,   2.02111,   3.52290],
 [ -1.81206,   1.37010,   3.91266],
  [-1.36039,   0.82134,   3.49370],
  [-1.23285,   0.92355,   2.68512],
  [-1.55698,   1.57456,   2.29536],
  [-1.33903,   1.53045,   1.50082],
  [-0.81464,   0.47720,   2.13140],
  [-0.88026,   0.85225,   1.39957],
  [-0.54584,   0.59443,   0.69083],
  [-0.65931,   1.02926,  -0.00106],
  [-1.10720,   1.72192,   0.01579],
  [-0.34231,   0.80587,  -0.72927]])

print "Coordinate dimensions: ", rough.shape,'\n'

Coordinate dimensions:  (13, 3) 



In [5]:
# find center of volume
centroid = np.mean(rough,axis=0)

# translate center volume to origin
scaled = np.add(rough,-centroid)

print "Center of original coordinates: \n", centroid,'\n'
print "Center of translated coordinates: \n", np.mean(scaled,axis=0),'\n'
print scaled

Center of original coordinates: 
[-1.21505154  1.21887154  1.81785   ] 

Center of translated coordinates: 
[ -8.54017711e-17  -2.64745490e-16   3.41607085e-17] 

[[-0.79355846  0.90441846  0.89638   ]
 [-0.92113846  0.80223846  1.70505   ]
 [-0.59700846  0.15122846  2.09481   ]
 [-0.14533846 -0.39753154  1.67585   ]
 [-0.01779846 -0.29532154  0.86727   ]
 [-0.34192846  0.35568846  0.47751   ]
 [-0.12397846  0.31157846 -0.31703   ]
 [ 0.40041154 -0.74167154  0.31355   ]
 [ 0.33479154 -0.36662154 -0.41828   ]
 [ 0.66921154 -0.62444154 -1.12702   ]
 [ 0.55574154 -0.18961154 -1.81891   ]
 [ 0.10785154  0.50304846 -1.80206   ]
 [ 0.87274154 -0.41300154 -2.54712   ]]


In [6]:
from sklearn.decomposition import PCA

# initialize and train PCA model on original coordinates
# get top 2 principal components of the coordinate matrix
pca = PCA(n_components=2)
reduced = pca.fit_transform(scaled)

# get matrix of the pca eigenvectors
principle_comps = pca.components_
print "Transformation matrix: \n", principle_comps,'\n'
print "Matrix dimensions: ", principle_comps.shape,'\n'

# check that vectors are normalized
print np.dot(principle_comps,principle_comps.T)

Transformation matrix: 
[[ 0.31031726 -0.13361454 -0.94119623]
 [ 0.47152379 -0.83806293  0.27443731]] 

Matrix dimensions:  (2, 3) 

[[  1.00000000e+00  -1.66533454e-16]
 [ -1.66533454e-16   1.00000000e+00]]


In [7]:
# get the angle between two vectors, component 1 from PCA, and x-axis (1,0,0)
# but really we're getting sin(theta/2)
# https://en.wikipedia.org/wiki/Euler%E2%80%93Rodrigues_formula
u=principle_comps[0,:]
print "component 1: ",u,"\n"
v=np.array([1,0,0])
sinHalfAng = math.sqrt((1-np.dot(u,v))/2)

# get axis perpendicular to x, u to use as axis of rotation
xaxis = -np.cross(u,v)
xaxis /= math.sqrt(np.dot(xaxis,xaxis)) # normalize
bcd = xaxis * sinHalfAng
print np.dot(bcd,bcd)
print "bcd: ",bcd,"\n"

# perform rotation
scaledx = rotate_frag(scaled, bcd)
tmatx = rotate_frag(principle_comps, bcd)
print tmatx

component 1:  [ 0.31031726 -0.13361454 -0.94119623] 

0.344841367698
bcd:  [-0.          0.58140258 -0.08253734] 

aa is 0.655159 

a is 0.809419 

aa is 0.655159 

a is 0.809419 

[[  1.00000000e+00   2.77555756e-17   1.66533454e-16]
 [ -2.77555756e-16  -7.89981116e-01   6.13131174e-01]]


In [11]:
# now do all the same with y-axis (0,1,0)

# get rotation angle
u=tmatx[1,:]
#u=tmatx[1,:]
print "component 2: ",u,"\n"
v=np.array([0,1,0])
sinHalfAng = math.sqrt((1-np.dot(u,v))/2)

# get y axis of rotation
yaxis = -np.cross(u,v)
yaxis /= math.sqrt(np.dot(yaxis,yaxis)) # normalize
bcd = yaxis * sinHalfAng
print np.dot(bcd,bcd)
print "bcd: ",bcd,"\n"

# perform rotation
scaledxy = rotate_frag(scaledx, bcd)
tmatxy = rotate_frag(tmatx, bcd)
print tmatxy

component 2:  [ -2.77555756e-16  -7.89981116e-01   6.13131174e-01] 

0.894990557907
bcd:  [  9.46039406e-01  -0.00000000e+00   4.28258575e-16] 

aa is 0.105009 

a is 0.324052 

aa is 0.105009 

a is 0.324052 

[[  1.00000000e+00  -1.97375285e-16   6.61722883e-16]
 [ -4.93038066e-31   1.00000000e+00   8.88178420e-16]]


In [12]:
print scaled.shape
print scaled,'\n'
print scaledx,'\n'
print scaledxy,'\n'

(13, 3)
[[-0.79355846  0.90441846  0.89638   ]
 [-0.92113846  0.80223846  1.70505   ]
 [-0.59700846  0.15122846  2.09481   ]
 [-0.14533846 -0.39753154  1.67585   ]
 [-0.01779846 -0.29532154  0.86727   ]
 [-0.34192846  0.35568846  0.47751   ]
 [-0.12397846  0.31157846 -0.31703   ]
 [ 0.40041154 -0.74167154  0.31355   ]
 [ 0.33479154 -0.36662154 -0.41828   ]
 [ 0.66921154 -0.62444154 -1.12702   ]
 [ 0.55574154 -0.18961154 -1.81891   ]
 [ 0.10785154  0.50304846 -1.80206   ]
 [ 0.87274154 -0.41300154 -2.54712   ]] 

[[-1.21076782  0.70003504 -0.54332045]
 [-1.99782253  0.5045887  -0.3916293 ]
 [-2.17709564 -0.13165008  0.10218089]
 [-1.56928875 -0.57237402  0.44423937]
 [-0.78233518 -0.37691207  0.29253685]
 [-0.60306207  0.25932671 -0.20127334]
 [ 0.21828337  0.32119484 -0.24929112]
 [-0.07175937 -0.7081585   0.54961969]
 [ 0.54656112 -0.27674901  0.21479248]
 [ 1.35184934 -0.41835167  0.32470085]
 [ 1.90974229  0.06179664 -0.04796173]
 [ 1.66234569  0.68355748 -0.53053362]
 [ 2.72334953 